# Figure experimental / scratch space

In [127]:
import sys
sys.path.append('/g/data/mn51/users/jb6465/drought-github/submodules/gwls')
from gwl import get_GWL_timeslice, get_GWL_syear_eyear
sys.path.append('/g/data/mn51/users/jb6465/drought-github')
import utils
sys.path.append('/g/data/mn51/users/jb6465/drought-github/submodules/plotting_maps')
from acs_area_statistics import acs_regional_stats, get_regions
from acs_plotting_maps import *
import glob
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rio
import geopandas
from shapely.geometry import mapping
from datetime import datetime

In [128]:
GWL=3.0
AGCD_MASK_SWITCH = True

In [129]:
if AGCD_MASK_SWITCH:
    mask_ds = xr.open_dataset("/g/data/mn51/users/dh4185/agcd_v1-0-2_precip_weight_r005_daily_1960_2022fraction_ge1.nc")
    q_mask = np.ma.masked_greater(mask_ds.fraction,0.8)
    # mask_ds.fraction.where(q_mask.mask).plot()

In [130]:
files = []
files.extend(sorted(glob.glob("/g/data/ia39/ncra/drought_aridity/spi/downscaled_BC_5km/QME-AGCD/full_period/*.nc")))
files.extend(sorted(glob.glob("/g/data/ia39/ncra/drought_aridity/spi/downscaled_BC_5km/MRNBC-AGCD/full_period/*.nc")))

In [131]:
%%time
plot_dict={}
for file in files:
    GCM = file.split('_')[6]
    RCM = 'BARPA-R' if file.split('_')[-4] == 'BOM' else 'CCAM-v2203-SN' if file.split('_')[-4] == 'CSIRO' else file.split('_')[-4]
    BC = file.split('_')[-1].split('-')[1]
    variant_id = file.split('_')[8]
    print(f'{GCM}_{variant_id}_{RCM}_{BC}')
    
    target_xr = xr.open_dataset(file)['SPI3']
    syear, eyear = get_GWL_syear_eyear('CMIP6',GCM,variant_id,'ssp370',GWL=GWL) 
    GWLx_xr = target_xr.sel(time=target_xr['time'].dt.year.isin(range(syear, eyear+1))) #have to do it this way as get_GWL_timeslice throws errors with 360 day calendars
    syear, eyear = get_GWL_syear_eyear('CMIP6',GCM,variant_id,'ssp370',GWL='1.2')
    GWL12_xr = target_xr.sel(time=target_xr['time'].dt.year.isin(range(syear, eyear+1)))
    
    GWLx_pct_time_below = 100*((GWLx_xr)<=-1).sum(dim='time')/len(GWLx_xr.time)
    GWL12_pct_time_below = 100*((GWL12_xr)<=-1).sum(dim='time')/len(GWL12_xr.time)
    
    plot_dict[f'{GCM}_{variant_id}_{RCM}_{BC}']=100*((GWLx_pct_time_below-GWL12_pct_time_below)/GWL12_pct_time_below).where(q_mask.mask) if AGCD_MASK_SWITCH==True else 100*((GWLx_pct_time_below-GWL12_pct_time_below)/GWL12_pct_time_below)

ACCESS-CM2_r4i1p1f1_BARPA-R_QME
ACCESS-CM2_r4i1p1f1_CCAM-v2203-SN_QME
ACCESS-ESM1-5_r6i1p1f1_BARPA-R_QME
ACCESS-ESM1-5_r6i1p1f1_CCAM-v2203-SN_QME
ACCESS-ESM1-5_r6i1p1f1_NARCliM2-0-WRF412R5_QME
CESM2_r11i1p1f1_BARPA-R_QME
CESM2_r11i1p1f1_CCAM-v2203-SN_QME
CMCC-ESM2_r1i1p1f1_BARPA-R_QME
CMCC-ESM2_r1i1p1f1_CCAM-v2203-SN_QME
CNRM-CM6-1-HR_r1i1p1f2_UQ-DES_QME
CNRM-ESM2-1_r1i1p1f2_CCAM-v2203-SN_QME
EC-Earth3-Veg_r1i1p1f1_NARCliM2-0-WRF412R5_QME
EC-Earth3_r1i1p1f1_BARPA-R_QME
EC-Earth3_r1i1p1f1_CCAM-v2203-SN_QME
FGOALS-g3_r4i1p1f1_UQ-DES_QME
GISS-E2-1-G_r2i1p1f2_UQ-DES_QME
MPI-ESM1-2-HR_r1i1p1f1_BARPA-R_QME
MPI-ESM1-2-HR_r1i1p1f1_NARCliM2-0-WRF412R5_QME
MRI-ESM2-0_r1i1p1f1_UQ-DES_QME
NorESM2-MM_r1i1p1f1_BARPA-R_QME
NorESM2-MM_r1i1p1f1_CCAM-v2203-SN_QME
NorESM2-MM_r1i1p1f1_UQ-DES_QME
UKESM1-0-LL_r1i1p1f2_NARCliM2-0-WRF412R3_QME
UKESM1-0-LL_r1i1p1f2_NARCliM2-0-WRF412R5_QME
ACCESS-CM2_r4i1p1f1_BARPA-R_MRNBC
ACCESS-CM2_r4i1p1f1_CCAM-v2203-SN_MRNBC
ACCESS-ESM1-5_r6i1p1f1_BARPA-R_MRNBC
ACCESS-ESM1-

In [154]:
df = pd.DataFrame({key: [value.mean().values] for key, value in plot_dict.items()})


In [155]:
df

,ACCESS-CM2_r4i1p1f1_BARPA-R_QME,ACCESS-CM2_r4i1p1f1_CCAM-v2203-SN_QME,ACCESS-ESM1-5_r6i1p1f1_BARPA-R_QME,ACCESS-ESM1-5_r6i1p1f1_CCAM-v2203-SN_QME,ACCESS-ESM1-5_r6i1p1f1_NARCliM2-0-WRF412R5_QME,CESM2_r11i1p1f1_BARPA-R_QME,CESM2_r11i1p1f1_CCAM-v2203-SN_QME,CMCC-ESM2_r1i1p1f1_BARPA-R_QME,CMCC-ESM2_r1i1p1f1_CCAM-v2203-SN_QME,CNRM-CM6-1-HR_r1i1p1f2_UQ-DES_QME,...,MPI-ESM1-2-HR_r1i1p1f1_NARCliM2-0-WRF412R3_MRNBC,MPI-ESM1-2-HR_r1i1p1f1_NARCliM2-0-WRF412R5_MRNBC,MRI-ESM2-0_r1i1p1f1_UQ-DES_MRNBC,NorESM2-MM_r1i1p1f1_BARPA-R_MRNBC,NorESM2-MM_r1i1p1f1_CCAM-v2203-SN_MRNBC,NorESM2-MM_r1i1p1f1_NARCliM2-0-WRF412R3_MRNBC,NorESM2-MM_r1i1p1f1_NARCliM2-0-WRF412R5_MRNBC,NorESM2-MM_r1i1p1f1_UQ-DES_MRNBC,UKESM1-0-LL_r1i1p1f2_NARCliM2-0-WRF412R3_MRNBC,UKESM1-0-LL_r1i1p1f2_NARCliM2-0-WRF412R5_MRNBC
0,5.364859131060931,-9.347145789890794,89.81937129433275,96.59205303724193,77.22369156453226,-7.5440216362546035,10.749231579464379,13.474277436298188,31.575734760534278,11.630274346566356,...,33.54593016183181,36.91049962565435,20.117213435325542,45.59851324141487,17.582231133007255,38.72365310089583,40.851685673109706,0.9052075012209289,34.57085594712393,39.660012456113776


In [156]:
groups = {
    "BOM": [col for col in df.columns if "BARPA" in col],
    "CSIRO": [col for col in df.columns if "CCAM" in col],
    "NSW": [col for col in df.columns if "NAR" in col],
    "QLD": [col for col in df.columns if "UQ" in col],
}


In [167]:
groups

{'BOM': ['ACCESS-CM2_r4i1p1f1_BARPA-R_QME',
  'ACCESS-ESM1-5_r6i1p1f1_BARPA-R_QME',
  'CESM2_r11i1p1f1_BARPA-R_QME',
  'CMCC-ESM2_r1i1p1f1_BARPA-R_QME',
  'EC-Earth3_r1i1p1f1_BARPA-R_QME',
  'MPI-ESM1-2-HR_r1i1p1f1_BARPA-R_QME',
  'NorESM2-MM_r1i1p1f1_BARPA-R_QME',
  'ACCESS-CM2_r4i1p1f1_BARPA-R_MRNBC',
  'ACCESS-ESM1-5_r6i1p1f1_BARPA-R_MRNBC',
  'CESM2_r11i1p1f1_BARPA-R_MRNBC',
  'CMCC-ESM2_r1i1p1f1_BARPA-R_MRNBC',
  'EC-Earth3_r1i1p1f1_BARPA-R_MRNBC',
  'MPI-ESM1-2-HR_r1i1p1f1_BARPA-R_MRNBC',
  'NorESM2-MM_r1i1p1f1_BARPA-R_MRNBC'],
 'CSIRO': ['ACCESS-CM2_r4i1p1f1_CCAM-v2203-SN_QME',
  'ACCESS-ESM1-5_r6i1p1f1_CCAM-v2203-SN_QME',
  'CESM2_r11i1p1f1_CCAM-v2203-SN_QME',
  'CMCC-ESM2_r1i1p1f1_CCAM-v2203-SN_QME',
  'CNRM-ESM2-1_r1i1p1f2_CCAM-v2203-SN_QME',
  'EC-Earth3_r1i1p1f1_CCAM-v2203-SN_QME',
  'NorESM2-MM_r1i1p1f1_CCAM-v2203-SN_QME',
  'ACCESS-CM2_r4i1p1f1_CCAM-v2203-SN_MRNBC',
  'ACCESS-ESM1-5_r6i1p1f1_CCAM-v2203-SN_MRNBC',
  'CESM2_r11i1p1f1_CCAM-v2203-SN_MRNBC',
  'CMCC-ESM2_r1i1p

In [163]:
plot_data = pd.DataFrame([
    {"RCM": group_name, "Value": val}
    for group_name, columns in groups.items()
    for column in columns
    for val in df[column].values  # Ensure scalars are used
])



In [164]:
plot_data

,Group,Value
0,BOM,5.364859131060931
1,BOM,89.81937129433275
2,BOM,-7.5440216362546035
3,BOM,13.474277436298188
4,BOM,-14.39124700759868
5,BOM,21.509310286634044
6,BOM,44.60152488270667
7,BOM,3.3196236374695403
8,BOM,87.81807566186626
9,BOM,-9.59595962817232
